In [2]:
import os
import numpy as np
# import pandas as pd
from matplotlib import pyplot as plt
import cv2
import time
import json
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, ReLU, Dropout, GRU, Bidirectional
#, ConvLSTM2D, Conv3D, Flatten
from tensorflow.keras.utils import to_categorical
import random 
from sklearn.model_selection import train_test_split
import mediapipe as mp
import pickle as pk

In [7]:
path ="../TrainData/"
file_list = os.listdir(path)

files = file_list[0:32]
words = files

In [8]:
np.unique(words)

array(['age', 'book', 'call', 'car', 'day', 'egypt', 'english', 'enjoy',
       'every', 'excuse', 'football', 'forget', 'fun', 'good', 'hate',
       'have', 'hello', 'help', 'holiday', 'iam', 'love', 'meet', 'month',
       'morning', 'my'], dtype='<U8')

In [9]:
formLableToNum = {label:num for num, label in enumerate(words)}
formNumTOLabel = {num:label for num, label in enumerate(words)}
formLableToNum

{'age': 0,
 'book': 1,
 'call': 2,
 'car': 3,
 'day': 4,
 'egypt': 5,
 'english': 6,
 'enjoy': 7,
 'every': 8,
 'excuse': 9,
 'football': 10,
 'forget': 11,
 'fun': 12,
 'good': 13,
 'hate': 14,
 'have': 15,
 'hello': 16,
 'help': 17,
 'holiday': 18,
 'iam': 19,
 'love': 20,
 'meet': 21,
 'month': 22,
 'morning': 23,
 'my': 24}

In [10]:
vids=[]
labels=[]
vidsnum = 0
for w in words:
    vidpath = path+w
    vidlist = os.listdir(vidpath)
    print(w,str(len(vidlist)))
    for i in vidlist:
        vid = []
        labels.append(formLableToNum[w])
        vidsnum+=1
        for frame_num in range(20):
            arr = np.load(os.path.join(path, w, i, "{}.npy".format(frame_num)))
            vid.append(arr)
        vid = np.reshape(np.array(vid),(20,258))
        # m = vid.mean(axis=0)
        # vid -=  m
        m = vid.mean(axis=-1)
        tmpv = vid.T 
        tmpv -=  m
        v = tmpv.T
        vids.append(v)
        
    
labels = np.reshape(np.array(labels),(-1,1))

age 63
book 59
call 65
car 61
day 60
egypt 65
english 64
enjoy 63
every 65
excuse 68
football 58
forget 66
fun 69
good 70
hate 59
have 59
hello 68
help 72
holiday 59
iam 62
love 60
meet 71
month 62
morning 81
my 62


In [13]:
vids = np.reshape(np.array(vids),(vidsnum,20,258))

In [14]:
X_train, X_dev, y_train, y_dev = train_test_split(vids, labels, test_size = 0.2,shuffle=True,random_state=10)
X_train.shape,y_train.shape


((1288, 20, 258), (1288, 1))

In [51]:
del vids
del labels

In [119]:
model_lstm_1 = Sequential([

tf.keras.Input(shape=(20,258),),
    
    
# GRU(64, return_sequences=True, activation='tanh', recurrent_dropout=0),
# Dropout(0.2, noise_shape =  (1,20, 1)),

# Bidirectional(LSTM(32, return_sequences=True, activation='relu'),input_shape=(20,258)),#, recurrent_dropout=0.2
# Dropout(0.4, noise_shape =  (1,20, 1)),
LSTM(32, return_sequences=True, activation='tanh'),
LSTM(64, return_sequences=True, activation='tanh'),       
LSTM(256, return_sequences=False, activation='tanh'),

BatchNormalization(),
Dropout(0.6),


Dense(128, activation='linear',kernel_regularizer=tf.keras.regularizers.L1L2(0.01,2)),#kernel_regularizer=tf.keras.regularizers.L2(0)
BatchNormalization(),#axis=-1,center=True,scale=True,
ReLU(), 

Dense(64, activation='linear',kernel_regularizer=tf.keras.regularizers.L1L2(0.05,8)),#kernel_regularizer=tf.keras.regularizers.L2(0)
BatchNormalization(),#axis=-1,center=True,scale=True,
ReLU(),    



Dense(25, activation='softmax')
])


In [120]:
model_lstm_1.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_47 (LSTM)              (None, 20, 32)            37248     
                                                                 
 lstm_48 (LSTM)              (None, 20, 64)            24832     
                                                                 
 lstm_49 (LSTM)              (None, 256)               328704    
                                                                 
 batch_normalization_48 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 dropout_41 (Dropout)        (None, 256)               0         
                                                                 
 dense_54 (Dense)            (None, 128)               32896     
                                                     

In [121]:
model_lstm_1.compile(tf.keras.optimizers.Adam(learning_rate=5e-4,beta_1=0.9,beta_2=0.999,epsilon=1e-07,)
        ,loss=tf.keras.losses.SparseCategoricalCrossentropy()
            ,metrics=["accuracy"],)#

In [122]:
model_lstm_1.fit(X_train, y_train, epochs=300, batch_size=128)#batch_size=64, validation_data=(X_dev, y_dev)

Epoch 1/512
11/11 [==============================] - 13s 385ms/step - loss: 1032.6439 - accuracy: 0.0598
Epoch 2/512
11/11 [==============================] - 4s 390ms/step - loss: 927.8524 - accuracy: 0.0738
Epoch 3/512
11/11 [==============================] - 4s 384ms/step - loss: 832.2668 - accuracy: 0.1064
Epoch 4/512
11/11 [==============================] - 4s 383ms/step - loss: 745.6214 - accuracy: 0.1359
Epoch 5/512
11/11 [==============================] - 4s 404ms/step - loss: 667.4821 - accuracy: 0.1786
Epoch 6/512
11/11 [==============================] - 5s 413ms/step - loss: 597.1599 - accuracy: 0.1879
Epoch 7/512
11/11 [==============================] - 4s 361ms/step - loss: 533.8422 - accuracy: 0.2189
Epoch 8/512
11/11 [==============================] - 4s 346ms/step - loss: 477.0205 - accuracy: 0.2275
Epoch 9/512
11/11 [==============================] - 4s 397ms/step - loss: 425.9354 - accuracy: 0.2578
Epoch 10/512
11/11 [==============================] - 4s 329ms/step - l

KeyboardInterrupt: 

In [123]:
model_lstm_1.evaluate(X_train, y_train)

41/41 [==============================] - 7s 100ms/step - loss: 0.3540 - accuracy: 1.0000


[0.3539767861366272, 1.0]

In [124]:
model_lstm_1.evaluate(X_dev, y_dev)

11/11 [==============================] - 1s 81ms/step - loss: 0.5968 - accuracy: 0.9412


[0.596828818321228, 0.9411764740943909]

***

In [118]:
model_lstm_1.save("./modelAsl/v2_")

INFO:tensorflow:Assets written to: ./modelAsl/v1_\assets


INFO:tensorflow:Assets written to: ./modelAsl/v1_\assets


****
****

In [ ]:
reconstructed_model = tf.keras.models.load_model("./modelAsl/v8")

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return  results

In [ ]:
def extract_keypoints(results):

    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
holistic = mp_holistic.Holistic(min_detection_confidence=0.25, min_tracking_confidence=0.25) 

In [ ]:
def videoProc(path, c=5):
    "###"
    cap = cv2.VideoCapture(path)
    success, frame = cap.read()
    
    fno = 0
    framecount = 20
    
    v = []
    
    skipCounter = c
    framerateCounter= round(fps/20, 0) 

    while True and success:
        success, frame = cap.read()
        if fno >=framecount:
            break 
        
     
        
        if skipCounter>=0:
            pass
        else:
            skipCounter-=1
            continue
        
        if fpsCounter==0:
            fpsCounter=framerateCounter-1
        else:
            fpsCounter-=1
            continue

        if success:
            frame = cv2.resize(frame, (720,480), interpolation = cv2.INTER_AREA)
            results = mediapipe_detection(frame, holistic)
            eres = extract_keypoints(results)

            tmp=  np.reshape(eres, (1,258))
            v.append(tmp)
            
            # draw_styled_landmarks(frame, results)
            # plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            # plt.show()
            fno += 1
        else:
            break

# if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
    cap.release()
    cv2.destroyAllWindows()
    
    f = len(v)
    v = np.array(v) if f >0 else[]
    v =np.reshape(v,(f,258))if f >0 else[]
   


    if f>0:      
        m = v.mean(axis=-1)     
        tmpv = v.T 
        tmpv -=  m
        v = tmpv.T
        m = v.mean(axis=0) 
        v -= m
    if fno<framecount and fno>0:
        tmp = np.zeros((framecount-fno,258))
        v = np.concatenate((v, tmp), axis=0) 

    return v

In [ ]:
v = videoProc(".\\TestDataDD\\z1.mp4",20)
# 1 0 3 2 4


(20) 16 (9) 7 4

In [ ]:
v.min(),v.max(),v.mean()

In [ ]:
p = reconstructed_model.predict_step(np.reshape(v, (1,20,258)))

In [ ]:
np.argmax(p),p

In [ ]:
reconstructed_model.summary()

In [92]:
from tensorflow import keras
from tensorflow.keras import layers

In [93]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [94]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(25, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [98]:
input_shape = X_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 20, 258)]    0           []                               
                                                                                                  
 layer_normalization_16 (LayerN  (None, 20, 258)     516         ['input_17[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 multi_head_attention_8 (MultiH  (None, 20, 258)     1060098     ['layer_normalization_16[0][0]', 
 eadAttention)                                                    'layer_normalization_16[0][0]'] 
                                                                                            

KeyboardInterrupt: 

In [ ]:
model_lstm_1.evaluate(X_train, y_train)